### 라이브러리 설치

In [ ]:
!pip install transformers>=4.45.0 newspaper3k==0.2.8 lxml_html_clean==0.4.1 gradio

### 카카오 kanana-nano-2.1b-instruct 써보기

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "kakaocorp/kanana-nano-2.1b-instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
).to("cuda")
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
from newspaper import Article

# Kanana 관련 뉴스
link = "https://www.aitimes.com/news/articleView.html?idxno=168364"
# GPT-4.5 관련 뉴스
link1 = "https://www.aitimes.com/news/articleView.html?idxno=168378"

article = Article(link)
article.download()
article.parse()
title = article.title
text = article.text.replace("\n\n", ' ')

In [ ]:
messages = [
    {"role": "system", "content": "당신은 뉴스 기사를 요약하는 역할을 수행합니다. 주어진 뉴스 기사를 공백을 포함하여 100자 정도로 요약해서 이해하기 쉽게 설명해주세요."},
    {"role": "user", "content": text}
]

input_ids = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
).to("cuda")


# 패딩 토큰 ID 설정
if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token  # eos_token이 보통 pad_token으로 사용됨

_ = model.eval()
with torch.no_grad():
    output = model.generate(
        input_ids,
        pad_token_id=tokenizer.pad_token_id,  # 패딩 토큰 ID 명시
        max_new_tokens=300,
        do_sample=False,
        return_dict_in_generate=True,  # 추가 옵션: generate 결과를 dict 형태로 반환
        output_scores=True  # 확률 점수 출력 옵션
    )

In [ ]:
# 프롬프트 이후 토큰만 추출
generated_ids = output.sequences[:, input_ids.size(1):]
result = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

print(result)

### OpenAI gpt-4.5-preview-2025-02-27 써보기

In [ ]:
!pip install openai

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "openai api key를 넣어주세요!"

In [ ]:
from openai import OpenAI

client = OpenAI()

completion = client.chat.completions.create(
    model="gpt-4.5-preview-2025-02-27",
    messages=[
        {"role": "system", "content": "당신은 뉴스 기사를 요약하는 역할을 수행합니다. 주어진 뉴스 기사를 공백을 포함하여 100자 정도로 요약해서 이해하기 쉽게 설명해주세요."},
        {
            "role": "user",
            "content": text
        }
    ]
)

In [ ]:
print(completion.choices[0].message.content)

### 뉴스 요약 비교기(kanana/gpt-4.5)

In [ ]:
import gradio as gr

# system_prompt = '당신은 뉴스 기사를 요약하는 역할을 수행합니다. 주어진 뉴스 기사를 공백을 포함하여 100자 정도로 요약해서 이해하기 쉽게 설명해주세요.'
system_prompt = '당신은 뉴스 기사를 영어로 번역하는 역할을 수행합니다. 주어진 뉴스 기사 내용만을 영어로 번역해주세요.'

# 요약 함수
def summarize_news(url, system_prompt):
    try:
        # 뉴스 크롤링
        article = Article(url)
        article.download()
        article.parse()
        title = article.title
        # text = article.text.replace("\n\n", ' ')
        text = article.text[100:200].replace("\n\n", ' ')

        # 모델 1 (카카오 Kanana 모델)
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": text}
        ]

        input_ids = tokenizer.apply_chat_template(
            messages,
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt"
        ).to("cuda")

        if tokenizer.pad_token_id is None:
            tokenizer.pad_token = tokenizer.eos_token

        _ = model.eval()
        with torch.no_grad():
            output = model.generate(
                input_ids,
                pad_token_id=tokenizer.pad_token_id,
                max_new_tokens=300,
                do_sample=False,
                return_dict_in_generate=True,
                output_scores=True
            )

        generated_ids = output.sequences[:, input_ids.size(1):]
        kanana_result = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

        # 모델 2 (OpenAI GPT-4.5)
        completion = client.chat.completions.create(
            model="gpt-4.5-preview-2025-02-27",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": text}
            ]
        )
        gpt_result = completion.choices[0].message.content

        return title, kanana_result, gpt_result
    except Exception as e:
        return "", f"오류 발생: {e}", ""

# Gradio 인터페이스
interface = gr.Interface(
    fn=summarize_news,
    inputs=[gr.Textbox(label="뉴스 기사 URL을 입력해주세요."),
            gr.Textbox(label="시스템 프롬프트", value=system_prompt, lines=3)
            ],
    outputs=[
        gr.Textbox(label="기사 제목"),
        gr.Textbox(label="Kanana 요약"),
        gr.Textbox(label="GPT-4.5 요약"),
    ],
    title="📰 뉴스 요약 비교기",
    description="뉴스 기사 URL을 입력하면 Kanana와 GPT-4.5의 요약 결과를 비교합니다."
)

if __name__ == "__main__":
    interface.launch()